In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re

In [3]:
options = Options()
options.headless = True
driver = webdriver.Chrome('./chromedriver',options=options)

In [4]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,driver):
    driver.get(url)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

In [5]:
def scrape_dir_page(dir_url,driver):
    print ('-'*20,'Scraping directory page','-'*20)
    new_links = []
    new_base_url = 'https://www.vox.com/'
    soup = get_js_soup(dir_url,driver)
    for link_holder in soup.find_all('div', class_ = 'c-entry-box--compact'):
        rel_link = link_holder.find('a')['href'] #get url
#         print(rel_link)
        if rel_link.startswith("https://www.vox.com/"):
            new_links.append(rel_link)
    print ('-'*20,'Found {} news urls'.format(len(new_links)),'-'*20)
    return new_links

In [6]:
dir_url = 'https://www.vox.com/'
news_links = scrape_dir_page(dir_url,driver)

-------------------- Scraping directory page --------------------
-------------------- Found 47 news urls --------------------


In [7]:
def scrape_news(news_url,driver):
    soup = get_js_soup(news_url,driver)   
    title = soup.select("h1")[0]
    content = soup.find_all('p')
    content_str = ""
    for p in content:
        if p.has_attr('id'):
            content_str += p.text
    return title.contents[0], content_str

In [8]:
output = {}
tot_urls = len(news_links)
for i,link in enumerate(news_links):
    print ('-'*20,'Scraping news url {}/{}'.format(i+1,tot_urls),'-'*20)
    title,content = scrape_news(link,driver)
    output[title] = (content, link)

-------------------- Scraping news url 1/47 --------------------
-------------------- Scraping news url 2/47 --------------------
-------------------- Scraping news url 3/47 --------------------
-------------------- Scraping news url 4/47 --------------------
-------------------- Scraping news url 5/47 --------------------
-------------------- Scraping news url 6/47 --------------------
-------------------- Scraping news url 7/47 --------------------
-------------------- Scraping news url 8/47 --------------------
-------------------- Scraping news url 9/47 --------------------
-------------------- Scraping news url 10/47 --------------------
-------------------- Scraping news url 11/47 --------------------
-------------------- Scraping news url 12/47 --------------------
-------------------- Scraping news url 13/47 --------------------
-------------------- Scraping news url 14/47 --------------------
-------------------- Scraping news url 15/47 --------------------
-------------------

In [9]:
def write_url(lst,file_):
    with open(file_,'w') as f:
        for key, value in lst.items():
            f.write(key + ". " + value[1] + "\n")
news_urls_file = 'vox#1120urls.txt'
write_url(output,news_urls_file)

In [10]:
def write_content(lst,file_):
    with open(file_,'w') as f:
        for key, value in lst.items():
            f.write(key + ". " + value[0] + "\n")
news_contents_file = 'vox#1120.txt'
write_content(output,news_contents_file)